# Problema para resolver en sala
La empresa TechStore vende productos tecnológicos tanto en su e-commerce como en sus tiendas físicas.
El equipo de finanzas quiere saber:

1. Ingresos y utilidad bruta por producto

2. Top 3 productos más vendidos por cada canal

3. ¿Qué productos tienen margen bajo (unit_price – unit_cost < 35 %) y deben ser renegociados con proveedores?

In [2]:
# 1. Importa CSV, cargar archivos y revisarlos como estimen convenientes
import pandas as pd

catalogo_df = pd.read_csv('techstore_catalogo.csv')
ventas_online_df = pd.read_csv('techstore_ventas_online.csv')
ventas_tienda_df = pd.read_csv('techstore_ventas_tienda.csv')

print(ventas_online_df.head(5))
print(ventas_tienda_df.head(5))

  order_id  order_date  product_id  quantity channel
0  ON00001  2025-06-13        1006         2  Online
1  ON00002  2025-06-27        1001         1  Online
2  ON00003  2025-06-12        1009         1  Online
3  ON00004  2025-06-30        1010         1  Online
4  ON00005  2025-06-02        1009         2  Online
  order_id  order_date  product_id  quantity channel
0  ST00001  2025-06-07        1005         5   Store
1  ST00002  2025-06-13        1006         1   Store
2  ST00003  2025-06-17        1005         2   Store
3  ST00004  2025-06-21        1009         2   Store
4  ST00005  2025-06-02        1002         4   Store


In [3]:
# 2. Unificar ventas de los 2 canales, utilizando metoodo MERGE. Técnica a realizar, utilizar el techstore_catalogo.csv como pivot entre las planillas
ventas_tienda_catalogo = pd.merge(ventas_tienda_df,catalogo_df,how='left', on='product_id')
ventas_online_catalogo = pd.merge(ventas_online_df,catalogo_df,how='left', on='product_id')
ventas_unificadas_df = pd.concat([ventas_tienda_catalogo,ventas_online_catalogo],ignore_index=True)

ventas_unificadas_df

,order_id,order_date,product_id,quantity,channel,product_name,category,unit_cost,unit_price
0,ST00001,2025-06-07,1005,5,Store,Tablet Plus,Tablets,979000,1605000
1,ST00002,2025-06-13,1006,1,Store,Tablet Mini,Tablets,611000,1040000
2,ST00003,2025-06-17,1005,2,Store,Tablet Plus,Tablets,979000,1605000
3,ST00004,2025-06-21,1009,2,Store,Monitor UltraWide,Monitores,798000,1152000
4,ST00005,2025-06-02,1002,4,Store,Laptop Air 13,Computadores,596000,796000
...,...,...,...,...,...,...,...,...,...
795,ON00396,2025-06-09,1005,3,Online,Tablet Plus,Tablets,979000,1605000
796,ON00397,2025-06-12,1010,3,Online,Teclado Mecánico,Periféricos,935000,1641000
797,ON00398,2025-06-07,1005,3,Online,Tablet Plus,Tablets,979000,1605000
798,ON00399,2025-06-06,1008,2,Online,Smartwatch Z,Accesorios,585000,780000


In [4]:
# 3. Agregar columna ingreso y utilidad bruta

ventas_unificadas_df['ingreso'] = ventas_unificadas_df['unit_price'] * ventas_unificadas_df['quantity']
ventas_unificadas_df['utilidad_bruta'] = (ventas_unificadas_df['unit_price'] - ventas_unificadas_df['unit_cost']) * ventas_unificadas_df['quantity']

ventas_unificadas_df

,order_id,order_date,product_id,quantity,channel,product_name,category,unit_cost,unit_price,ingreso,utilidad_bruta
0,ST00001,2025-06-07,1005,5,Store,Tablet Plus,Tablets,979000,1605000,8025000,3130000
1,ST00002,2025-06-13,1006,1,Store,Tablet Mini,Tablets,611000,1040000,1040000,429000
2,ST00003,2025-06-17,1005,2,Store,Tablet Plus,Tablets,979000,1605000,3210000,1252000
3,ST00004,2025-06-21,1009,2,Store,Monitor UltraWide,Monitores,798000,1152000,2304000,708000
4,ST00005,2025-06-02,1002,4,Store,Laptop Air 13,Computadores,596000,796000,3184000,800000
...,...,...,...,...,...,...,...,...,...,...,...
795,ON00396,2025-06-09,1005,3,Online,Tablet Plus,Tablets,979000,1605000,4815000,1878000
796,ON00397,2025-06-12,1010,3,Online,Teclado Mecánico,Periféricos,935000,1641000,4923000,2118000
797,ON00398,2025-06-07,1005,3,Online,Tablet Plus,Tablets,979000,1605000,4815000,1878000
798,ON00399,2025-06-06,1008,2,Online,Smartwatch Z,Accesorios,585000,780000,1560000,390000


In [5]:
# 4. Buscar los 3 productos mas vendidos por cada canal

ventas_agrupadas = ventas_unificadas_df.groupby(['channel', 'product_id','product_name'])['quantity'].sum().reset_index()
ventas_agrupadas.sort_values(['channel', 'quantity'], ascending=[True, False],inplace=True)

ventas_agrupadas.groupby('channel').head(3).set_index('channel')

,product_id,product_name,quantity
channel,,,
Online,1004,Smartphone Lite,147
Online,1002,Laptop Air 13,134
Online,1008,Smartwatch Z,133
Store,1005,Tablet Plus,137
Store,1007,Auriculares NoiseX,133
Store,1001,Laptop Pro 14,131


In [9]:
#  Calcular el margen inferior al solicitado para renegociar con proveedores
ventas_unificadas_df['margen_inferior'] = ((ventas_unificadas_df['unit_price'] - ventas_unificadas_df['unit_cost'])/ventas_unificadas_df['unit_price']).round(2)
productos_margen_bajo = ventas_unificadas_df[ventas_unificadas_df['margen_inferior'] < 0.35]

list(productos_margen_bajo['product_name'].unique())

['Monitor UltraWide',
 'Laptop Air 13',
 'Smartphone X',
 'Smartwatch Z',
 'Auriculares NoiseX']